In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

import pathlib
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
with open('./dataset/fungi/train_images_cat.txt', 'r') as f:
    train_images_cat = f.readlines()
f.close()
with open('./dataset/fungi/val_images_cat.txt', 'r') as f:
    val_images_cat = f.readlines()
f.close()
len(train_images_cat), len(val_images_cat)

In [ ]:
train_images_cat[0]

In [ ]:
train_images_total = [i.split('@:@')[1].strip().strip('\n') for i in train_images_cat]
train_cats_total = [i.split('@:@')[0].strip().strip('\n') for i in train_images_cat]
train_images_total[0], train_cats_total[0]
# train_images = [
#     "./dataset/fungi/train/images/" + i.split('@:@')[-1].strip('\n') for i in train_images_cat
# ]
train_images = [str(i) for i in pathlib.Path('./dataset/fungi/train/').glob('**/*.JPG')]
# train_images[:3]
train_lbls = [
    i.split('@:@')[0].strip('\n') for i in train_images_cat
]
# train_lbls[:3]
len(train_images), len(train_lbls)
# train_images_names[:3]
train_images_names = ["/".join(str(i).split('/')[-2:]) for i in pathlib.Path('./dataset/fungi/train/').glob('**/*.JPG')]
train_images_names[:3]

In [ ]:
df_full = pd.DataFrame([train_images_total, train_cats_total]).T
df_full.columns = ['file_name', 'label']
# df_full.head()
df_train = pd.DataFrame([train_images, train_lbls[:len(train_images)]]).T
df_train.columns = ['file_path', 'label']
df_train['label'] = df_train['label'].astype(int)
df_train['file_name'] = df_train['file_path'].apply(lambda x: "/".join(x.split('/')[-2:]))
# df_train.head()
df_train_real = pd.merge(left=df_full, right=df_train, on = 'file_name', how='inner')
df_train_real.drop(columns=['label_y'], inplace=True)
df_train_real['file_name'] = df_train_real['file_name'].apply(lambda x: x.split('/')[-1])
df_train_real.columns = ['file_name', 'label', 'file_path']
df_train_real['label_name'] = df_train_real['file_path'].apply(lambda x: x.split('/')[-2])
df_train_real['label'] = df_train_real['label'].astype(int)
df_train_real.shape

In [ ]:
val_images_total = [i.split('@:@')[1].strip().strip('\n') for i in val_images_cat]
val_cats_total = [i.split('@:@')[0].strip().strip('\n') for i in val_images_cat]
# val_images_total[0], train_cats_total[0]
val_images = [str(i) for i in pathlib.Path('./dataset/fungi/val/').glob('**/*.JPG')]
val_lbls = [i.split('@:@')[0].strip('\n') for i in train_images_cat]
val_images_names = ["/".join(str(i).split('/')[-2:]) for i in pathlib.Path('./dataset/fungi/val/').glob('**/*.JPG')]
len(val_images), len(val_lbls), val_images_names[:3]

In [ ]:
df_full = pd.DataFrame([val_images_total, val_cats_total]).T
df_full.columns = ['file_name', 'label']
# df_full.head()
df_val = pd.DataFrame([val_images, val_lbls[:len(val_images)]]).T
df_val.columns = ['file_path', 'label']
df_val['label'] = df_val['label'].astype(int)
df_val['file_name'] = df_val['file_path'].apply(lambda x: "/".join(x.split('/')[-2:]))
df_val_real = pd.merge(left=df_full, right=df_val, on = 'file_name', how='inner')
df_val_real.drop(columns=['label_y'], inplace=True)
df_val_real['file_name'] = df_val_real['file_name'].apply(lambda x: x.split('/')[-1])
df_val_real.columns = ['file_name', 'label', 'file_path']
df_val_real['label_name'] = df_val_real['file_path'].apply(lambda x: x.split('/')[-2])
df_val_real['label'] = df_val_real['label'].astype(int)
df_val_real.shape

In [ ]:
df_val_real.head()

In [ ]:
del df_train, df_val, df_full

In [ ]:
df_train_real.head()

In [ ]:
plt.imshow(Image.open(df_train_real.iloc[0,2]),);
# plt.legend(show=True)
plt.title(label=f"{str(df_train_real.iloc[0, 1])}/{str(df_train_real.iloc[0, -1])}")
plt.show()

In [ ]:
df_val_real[df_val_real['label']==650]

In [ ]:
plt.imshow(Image.open(df_val_real.iloc[0,2]),);
# plt.legend(show=True)
plt.title(label=str(df_val_real.iloc[0, 1]) + str(df_val_real.iloc[0, -1]))
plt.show()


In [ ]:
# import shutil
# import os
# # os.makedirs("./dataset/fungi/val", exist_ok=True)
# for src in tqdm(train_images[:]):
#     os.makedirs("/".join(src.replace('fungi_train_val/images', 'train').split('/')[:-1]), exist_ok=True)
#     shutil.copyfile(src, src.replace('fungi_train_val/images', 'train'))
# import shutil
# import os
# # os.makedirs("./dataset/fungi/val", exist_ok=True)
# for src in tqdm(val_images[:]):
#     os.makedirs("/".join(src.replace('fungi_train_val/images', 'val').split('/')[:-1]), exist_ok=True)
#     shutil.copyfile(src, src.replace('fungi_train_val/images', 'val'))

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.RandomRotation(degrees=(30, 70)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])
# the validation transforms
valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

BATCH_SIZE = 4
ROOT = "./dataset/fungi"
# training dataset
train_dataset = datasets.ImageFolder(
    root= f'{ROOT}/train',
    transform=train_transform
)
# validation dataset
valid_dataset = datasets.ImageFolder(
    root= f'{ROOT}/val',
    transform=valid_transform
)
# training data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers = 4, 
    pin_memory=True
)
# validation data loaders
valid_loader = DataLoader(
    valid_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    num_workers = 2, 
    pin_memory=True
)

In [ ]:
train_dataset.__len__()

In [ ]:
classes = sorted(df_train_real['label'].unique().tolist())
len(classes), classes[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)
print(labels)

In [ ]:
df_train_real[df_train_real['label']==26]

In [ ]:
# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5}' for j in range(BATCH_SIZE)))

In [ ]:
def save_model(epochs, model, optimizer, criterion):
    """
    Function to save the trained model to disk.
    """
    torch.save({
                'epoch': epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, 'outputs/model.pth')
def save_plots(train_acc, valid_acc, train_loss, valid_loss):
    """
    Function to save the loss and accuracy plots to disk.
    """
    # accuracy plots
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_acc, color='green', linestyle='-', 
        label='train accuracy'
    )
    plt.plot(
        valid_acc, color='blue', linestyle='-', 
        label='validataion accuracy'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('outputs/accuracy.png')
    
    # loss plots
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_loss, color='orange', linestyle='-', 
        label='train loss'
    )
    plt.plot(
        valid_loss, color='red', linestyle='-', 
        label='validataion loss'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('outputs/loss.png')

In [ ]:



# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


# net = Net()

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 5)
        
        # self.fc1 = nn.Linear(256, 50)
        self.fc1 = nn.Linear(256, 341)
        
        self.pool = nn.MaxPool2d(2, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        bs, _, _, _ = x.shape
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        x = self.fc1(x)
        return x

In [ ]:

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# for epoch in range(2):  # loop over the dataset multiple times

#     running_loss = 0.0
#     for i, data in enumerate(train_loader, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = net(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
#             running_loss = 0.0

# print('Finished Training')

lr = 1e-3
epochs = 2
# device = ('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(f"Computation device: {device}\n")
model = CNNModel().to(device)
print(model)
# total parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
import os
# os.environ['CUDA_DEVICE_AVAILABLE'] = '0'
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
t = torch.tensor([1,2], device='cpu')

In [ ]:
def train(model, trainloader, optimizer, criterion):
    model.train()
    print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        counter += 1
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # forward pass
        outputs = model(image)
        print(outputs.shape)
        # calculate the loss
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()
        # backpropagation
        loss.backward()
        # update the optimizer parameters
        optimizer.step()
    
    # loss and accuracy for the complete epoch
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc
# validation
def validate(model, testloader, criterion):
    model.eval()
    print('Validation')
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader), total=len(testloader)):
            counter += 1
            
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            # forward pass
            outputs = model(image)
            # calculate the loss
            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()
            # calculate the accuracy
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()
        
    # loss and accuracy for the complete epoch
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(testloader.dataset))
    return epoch_loss, epoch_acc

In [ ]:
len(train_dataset.classes), train_dataset.__len__()

In [ ]:
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []
# start the training
for epoch in range(epochs):
    print(f"[INFO]: Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_acc = train(model, train_loader, 
                                              optimizer, criterion)
    valid_epoch_loss, valid_epoch_acc = validate(model, valid_loader,  
                                                 criterion)
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    print(f"Training loss: {train_epoch_loss:.3f}, training acc: {train_epoch_acc:.3f}")
    print(f"Validation loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f}")
    print('-'*50)
    # time.sleep(5)
    
# save the trained model weights
# save_model(epochs, model, optimizer, criterion)
# save the loss and accuracy plots
# save_plots(train_acc, valid_acc, train_loss, valid_loss)
print('TRAINING COMPLETE')

In [ ]:
torch.cuda.is_available()

In [ ]:
data = torch.randn((3,10), device=torch.device("cuda"))